# 高级应用开发

In [1]:
import os
import warnings
from IPython.display import Markdown
from setenv import APIKeyManager

# 忽略警告
warnings.filterwarnings('ignore')
# 设置API密钥
key_manager = APIKeyManager("/root/.2233_langchain_config.json").setup_api_key(["OPENAI_API_KEY", "OPENAI_API_BASE"])

"""
# 卸载所有库
!pip uninstall -y numpy pandas scipy scikit-learn nltk unstructured langchain langchain-community  

# 安装指定版本
!pip install numpy==1.23.5  
!pip install pandas==1.5.3 scipy==1.10.1 scikit-learn==1.2.2  
!pip install nltk==3.8.1  
!pip install unstructured  
!pip install langchain langchain-community  
"""

# 查看Python版本
!python -V
# 查看安装的库
!pip list | grep -E 'langchain|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.9.21
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        7.5.3
langchain                                0.3.21
langchain-community                      0.3.20
langchain-core                           0.3.47
langchain-deepseek                       0.1.3
langchain-openai                         0.3.9
langchain-text-splitters                 0.3.7
numpy                                    1.23.5
openai                                   1.68.2
scipy                                    1.10.1
tiktoken                                 0.9.0
unstructured                             0.17.2
unstructured-client                      0.31.3


## 1. 知识库问答系统

### 1.1 文档处理

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [3]:
# 加载文档
# 先下载nltk的punkt资源
import nltk
# nltk.download('punkt')

# 然后加载文档
try:
    loader = DirectoryLoader(
        './docs',
        glob="**/*.{txt,pdf}"
    )
    documents = loader.load()
    print(f"成功加载了 {len(documents)} 个文档")
except Exception as e:
    print(f"Error loading file docs/sample.txt\n{str(e)}")
    documents = []

成功加载了 0 个文档


In [ ]:
# 分割文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

# 创建向量存储
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings
)

In [ ]:
# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 测试检索效果
query = "请介绍一下向量数据库的作用"
docs = retriever.get_relevant_documents(query)

# 打印检索结果
print(f"检索到 {len(docs)} 个相关文档")
for i, doc in enumerate(docs):
    print(f"\n--- 文档 {i+1} ---")
    print(f"内容: {doc.page_content[:200]}...")
    print(f"来源: {doc.metadata.get('source', '未知')}")


/tmp/ipykernel_13557/2615154315.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


检索到 1 个相关文档

--- 文档 1 ---
内容: 这是一个示例文档，用于演示知识库问答系统。...
来源: docs/sample.txt


## 2. 智能代理系统

### 2.1 任务规划

智能代理系统是LangChain的高级应用，它能够根据用户的指令自主规划和执行任务。
代理系统通常包含以下组件：
1. 任务规划：将复杂任务分解为可执行的步骤
2. 工具调用：使用各种工具完成特定任务
3. 记忆管理：保存执行过程中的状态和结果
4. 反思与优化：根据执行结果改进后续行动

下面我们将实现一个简单的任务规划代理，展示如何构建智能代理系统


In [9]:
from langchain.agents import Tool 
from langchain.prompts import PromptTemplate  # 导入提示模板
from langchain.llms import OpenAI  
from langchain_deepseek import ChatDeepSeek  
from langchain.chains import LLMChain  # 导入LLM链

# 重新设置为deepseek api
APIKeyManager().setup_api_key()

# 创建任务规划代理
class TaskPlanningAgent:
    def __init__(self, llm=None):
        self.llm = llm or OpenAI(temperature=0)  # 移除model参数，避免404错误
        self.tools = [
            Tool(
                name="Plan",
                func=self.plan_task,
                description="规划任务步骤"
            ),
            Tool(
                name="Execute",
                func=self.execute_task,
                description="执行任务"
            )
        ]
        
    def plan_task(self, task):
        """规划任务步骤"""
        template = """
        请为以下任务制定详细的执行计划，将任务分解为具体步骤：
        
        任务: {task}
        
        请列出完成此任务所需的步骤：
        """
        
        prompt = PromptTemplate(template=template, input_variables=["task"])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        plan = chain.invoke({"task": task})  # 使用run方法替代invoke方法
        return plan
        
    def execute_task(self, plan):
        """执行任务"""
        template = """
        请执行以下计划中的步骤，并报告执行结果：
        
        计划: {plan}
        
        执行结果：
        """
        
        prompt = PromptTemplate(template=template, input_variables=["plan"])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.invoke({"plan": plan})  # 使用run方法替代invoke方法
        return result

In [10]:
# 创建代理实例
planning_agent = TaskPlanningAgent(llm=ChatDeepSeek(temperature=0, model="deepseek-chat", api_key=os.environ["OPENAI_API_KEY"]))

# 测试任务规划功能
plan_result = planning_agent.plan_task("为团队组织一次线上技术分享会")

# 测试任务执行功能
execution_result = planning_agent.execute_task("1. 确定分享主题\n2. 邀请演讲者\n3. 安排时间和平台\n4. 发送邀请")

In [11]:
Markdown(plan_result['text'])

### 线上技术分享会执行计划

#### 1. 明确目标和主题
- **确定分享会目的**：提升团队技术能力、分享最新技术趋势、解决当前项目难点等。
- **选定主题**：根据团队需求和技术热点，确定1-3个具体技术主题（如“前端性能优化”、“微服务架构实践”）。
- **确认受众**：明确参与人员（如全团队、特定项目组、跨部门）及人数。

#### 2. 确定时间和形式
- **时间安排**：
  - 选择日期和时段（避开项目高峰期，预留1-2周准备时间）。
  - 时长建议：1-2小时（含Q&A）。
- **形式**：
  - 平台选择（Zoom/腾讯会议/Teams等）。
  - 互动方式：直播+录播、是否允许匿名提问、聊天室讨论等。

#### 3. 邀请分享嘉宾
- **内部或外部嘉宾**：
  - 内部：从团队中挑选有经验的成员。
  - 外部：联系行业专家或合作公司技术负责人。
- **沟通内容**：
  - 主题、时间、时长、受众背景。
  - 提供大纲模板（如“问题背景-解决方案-案例演示”）。
  - 确认是否需要试讲或材料审核。

#### 4. 会前准备
- **材料收集**：
  - 提前1周收取嘉宾的PPT、Demo代码或参考资料。
  - 统一格式（如PPT模板、公司LOGO）。
- **技术测试**：
  - 测试会议软件（音视频、屏幕共享、录制功能）。
  - 确保嘉宾网络稳定，备用方案（如录播视频）。
- **宣传与报名**：
  - 发送邮件/群通知，附主题简介、嘉宾介绍、参会链接。
  - 使用报名工具（如Google表单）统计人数。
- **日程细化**：
  - 制定详细流程（如：5分钟开场→40分钟分享→15分钟Q&A→总结）。

#### 5. 会中执行
- **主持人工作**：
  - 提前15分钟开启会议，调试设备。
  - 介绍规则（如“提问请用聊天框”“录音声明”）。
  - 控制时间，引导Q&A环节。
- **技术支持**：
  - 安排专人处理突发问题（如音画不同步、断网）。
  - 实时备份录制内容。

#### 6. 会后跟进
- **材料归档**：
  - 分享录制视频、PPT至团队知识库（如Confluence/GitHub）。
  - 整理Q&A高频问题，补充文档。
- **反馈收集**：
  - 发送匿名问卷（主题实用性、讲师表现、改进建议）。
- **致谢与激励**：
  - 公开感谢分享者（邮件/团队群）。
  - 考虑设立奖励机制（如“最佳分享奖”）。

#### 7. 持续优化
- **复盘会议**：
  - 分析参与率、反馈分数，总结不足（如时间过长、主题偏离需求）。
- **规划后续**：
  - 根据反馈制定下一期主题（如每月1次系列分享）。

---

**关键节点示例**：
- D-14：确定主题、嘉宾  
- D-7：完成材料收集，发送通知  
- D-1：技术测试，提醒参会  
- D-day：主持执行  
- D+1：归档材料，发送反馈表  

通过以上步骤，可确保分享会高效有序进行，同时积累团队知识资产。

In [12]:
Markdown(execution_result['text'])

### 执行结果：

1. **确定分享主题**  
   - 已确定本次分享的主题为 **“人工智能在行业中的应用与未来趋势”**。  
   - 主题范围涵盖技术发展、实际案例和未来方向，确保内容对目标受众（如技术从业者、企业管理者）具有吸引力。

2. **邀请演讲者**  
   - 成功邀请到以下两位专家作为演讲者：  
     - **张教授**（某大学人工智能实验室负责人，研究方向为机器学习与产业落地）。  
     - **李总监**（某科技公司AI产品负责人，实战经验丰富）。  
   - 两位演讲者均已确认档期，并对主题表示兴趣。

3. **安排时间和平台**  
   - 时间：活动定于 **2023年11月20日 14:00-16:00（北京时间）**。  
   - 平台：使用 **Zoom** 作为线上会议工具，并预留备用平台（腾讯会议）。  
   - 日程安排：  
     - 14:00-14:10 开场简介  
     - 14:10-15:00 张教授主题演讲  
     - 15:00-15:50 李总监案例分享  
     - 15:50-16:00 Q&A环节  

4. **发送邀请**  
   - 已通过邮件和内部通讯工具向目标受众发送邀请，覆盖约 **200人**。  
   - 邮件包含主题详情、时间、参与链接及演讲者介绍。  
   - 目前收到 **85人** 的确认回复，后续将跟进未回复者。  

**当前进展**：计划所有步骤均已完成，活动进入筹备执行阶段（如测试平台、准备材料）。  
**下一步**：活动前1天发送提醒邮件，确保参与率。  

---  
如需调整或补充细节（如扩大宣传、增加演讲者），请进一步指示。

### 2.2 工具调用

In [15]:

from langchain.agents import initialize_agent, Tool
from langchain_deepseek import ChatDeepSeek
# 创建自定义工具
def calculator(expression):
    return eval(expression)

tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="计算数学表达式"
    )
]

# 创建工具调用代理
agent = initialize_agent(
    tools,
    ChatDeepSeek(temperature=0, model="deepseek-chat", api_key=os.environ["OPENAI_API_KEY"]),
    agent="zero-shot-react-description",
    verbose=True
)

# 执行计算
result = agent.run("计算 23 * 45")



> Entering new AgentExecutor chain...
Thought: I need to calculate the product of 23 and 45.
Action: Calculator
Action Input: 23 * 45
Observation: 1035
Thought:I now know the final answer
Final Answer: 23 * 45 = 1035

> Finished chain.


In [14]:
Markdown(result)

23 * 45 = 1035

In [28]:
# 创建更复杂的工具集
from datetime import datetime

# 获取当前时间工具
def get_current_time(*args):
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 文本分析工具
def analyze_text(text):
    words = len(text.split())
    chars = len(text)
    return f"文本分析结果：{words}个单词，{chars}个字符"

# 将时间转换为中文
def get_time_cn(date_time: str):
    """将时间转换为中文"""
    # 去除可能存在的前后空格
    date_time = date_time.strip()
    _time = datetime.strptime(date_time, "%Y-%m-%d %H:%M:%S")
    
    # 定义数字到中文的映射
    num_to_cn = {
        '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
        '5': '五', '6': '六', '7': '七', '8': '八', '9': '九'
    }
    
    # 将年份中的数字转换为中文
    year = ''.join([num_to_cn[digit] for digit in str(_time.year)])
    
    # 将月、日、时、分、秒转换为中文（采用十进制表示法）
    def convert_to_cn(num):
        num = int(num)
        if num == 0:
            return '零'
        elif num < 10:
            return num_to_cn[str(num)]
        elif num < 20:
            return '十' + ('' if num == 10 else num_to_cn[str(num % 10)])
        else:
            return num_to_cn[str(num // 10)] + '十' + ('' if num % 10 == 0 else num_to_cn[str(num % 10)])
    
    month = convert_to_cn(_time.month)
    day = convert_to_cn(_time.day)
    hour = convert_to_cn(_time.hour)
    minute = convert_to_cn(_time.minute)
    second = convert_to_cn(_time.second)
    
    return f"{year}年{month}月{day}日 {hour}时{minute}分{second}秒"

# 创建复杂工具集
complex_tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="用于计算数学表达式，例如：1+2, 3*4, 10/2等"
    ),
    Tool(
        name="CurrentTime",
        func=get_current_time,
        description="获取当前的日期和时间"
    ),
    Tool(
        name="TextAnalyzer",
        func=analyze_text,
        description="分析文本，计算单词数和字符数"
    ),
    Tool(
        name="TimeCN",
        func=get_time_cn,
        description="将日期时间字符串转换为中文格式，输入格式应为'%Y-%m-%d %H:%M:%S'"
    )
]

# 创建复杂工具调用代理
complex_agent = initialize_agent(
    complex_tools,
    ChatDeepSeek(temperature=0, model="deepseek-chat", api_key=os.environ["OPENAI_API_KEY"]),
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True  # 添加错误处理参数
)

# 执行复杂任务
complex_result = complex_agent.run("先查询人工智能的信息，然后分析返回文本的长度，并获取当前时间，然后将时间转换为中文格式")



> Entering new AgentExecutor chain...
我需要先获取关于人工智能的信息，然后分析这段文本的长度，接着获取当前时间，最后将时间转换为中文格式。由于我没有直接获取人工智能信息的工具，我将跳过第一步，直接分析给定的文本长度，然后获取当前时间并转换为中文格式。

Action: CurrentTime
Action Input: None
Observation: 2025-03-25 08:47:11
Thought:现在我已经获取了当前时间，接下来需要将时间转换为中文格式。

Action: TimeCN
Action Input: 2025-03-25 08:47:11
Observation: 二零二五年三月二十五日 八时四十七分十一秒
Thought:现在我需要分析一段关于人工智能的文本的长度。由于没有具体的文本提供，我将假设一段示例文本进行分析。

Action: TextAnalyzer
Action Input: 人工智能是研究、开发用于模拟、延伸和扩展人的智能的理论、方法、技术及应用系统的一门新的技术科学。
Observation: 文本分析结果：1个单词，48个字符
Thought:现在我已经完成了所有要求的步骤：分析了文本长度，获取了当前时间并将其转换为中文格式。

Final Answer: 
1. 分析文本结果：示例文本"人工智能是研究、开发用于模拟、延伸和扩展人的智能的理论、方法、技术及应用系统的一门新的技术科学。"包含1个单词和48个字符
2. 当前中文时间：二零二五年三月二十五日 八时四十七分十一秒

> Finished chain.


In [29]:
Markdown(complex_result)

1. 分析文本结果：示例文本"人工智能是研究、开发用于模拟、延伸和扩展人的智能的理论、方法、技术及应用系统的一门新的技术科学。"包含1个单词和48个字符
2. 当前中文时间：二零二五年三月二十五日 八时四十七分十一秒